# NWB-Adapter (Simple)

This notebook demostrates how to use the new `dj-AttributeAdapter` feature to work with `NWB` objects.

In [ ]:
# import datajoint, nwb modules
%matplotlib inline
import datajoint as dj
import os
import pynwb
from pynwb import NWBFile, NWBHDF5IO
from datetime import datetime
from dateutil.tz import tzlocal
import json
import numpy as np
import pathlib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.environ['DJ_SUPPORT_ADAPTED_TYPES'] = 'TRUE'
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'

In [ ]:
dj.config['enable_python_native_blobs'] = True

# Objective

This notebook presents a specific example usecase of incorporating ***NWB*** objects into DataJoint using the new ***dj.AttributeAdapter*** feature. 

We wish to create a table storing NWB object, with attribute type of ***filepath***. The idea is to generate NWB files, one for each session, that can be access from the file system, hence type ***filepath***, and can also be fetched and worked with as part of DatJoint pipeline. This can be accomplished with the new ***dj.AttributeAdapter*** feature.

As prerequisite, readers should be familiar with the concept and operation of ***dj.AttributeAdapter***, a review can be found [here](./Adapted-Types.ipynb).

## Step 0 - Create a ***store*** for the filepath

In [ ]:
exported_nwb_dir = '/Users/shanshen/data/nwb_store'

In [ ]:
dj.config['stores'] = {
    'nwbstore': {'protocol': 'file',
                 'stage': exported_nwb_dir,
                 'location': exported_nwb_dir}}

## Step 1 - Create a DataJoint AttributeAdapter for NWB object

Basically we will need to define an object inhereted from `dj.AttributeAdapter` and instantiated with a variable name ***nwb_obj***

In [ ]:
class NWBFileAdapter(dj.AttributeAdapter):
    attribute_type = 'filepath@nwbstore'
    
    def put(self, nwb):
        save_file_name = ''.join([nwb.identifier, '.nwb'])
        # save the file
        with NWBHDF5IO(os.path.join(exported_nwb_dir, save_file_name), mode='w') as io:
            io.write(nwb)
            print(f'Write NWB 2.0 file: {save_file_name}')
        # return the filepath to be inserted into DataJoint tables
        return os.path.join(exported_nwb_dir, save_file_name)
        
    def get(self, path):
        # read the nwb filepath and return an nwb file object back to the user
        return NWBHDF5IO(path, mode='r').read()

#### Instantiate for use as a datajoint type

In [ ]:
nwb_file = NWBFileAdapter()

## Step 2 - Create a new schema ***export*** and NWB table

This ***NWB*** table specifies a primary key of `experiment.Session`, designed to store one NWB object (or NWBFile) per session

In [ ]:
schema = dj.schema('demo_nwb_adapter')

In [ ]:
@schema
class NWB(dj.Manual):
    definition = """
    nwb_id: int
    ---
    nwb: <nwb_file> 
    """

Note that the table definition above set the ***nwb*** attribute to be of type ***< nwb_obj >***. 

Hence the reason for defining ***nwbfile*** as an instant of ***NWBAdapter*** - see Step 1

## Step 3 - Build an NWBFile

Here, we build a very simple NWB object using the `pynwb` package, for the sake of demonstration

In [ ]:
# -- create NWB 
nwb = NWBFile(identifier='nwb_01',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')
# -- add subject
nwb.subject = pynwb.file.Subject(
    subject_id='animal_01',
    sex='F')

In [ ]:
nwb

## Step 4 -  Insert to the ***NWB*** table

In [ ]:
NWB()

In [ ]:
schema.external['nwbstore'].delete(delete_external_files=True)

In [ ]:
NWB.insert1({'nwb_id': 0, 'nwb': nwb})

In [ ]:
NWB()

### Now, fetch that NWB file back

In [ ]:
fetched_nwb = (NWB & 'nwb_id=0').fetch1('nwb')

In [ ]:
fetched_nwb

## This concludes the basic showcase of using `dj.AttributeAdapter` to work with `NWB` objects

Continue further to see more examples, but the core usage is demonstrated above

In [ ]:
# -- create NWB 
nwb2 = NWBFile(identifier='nwb_02',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')
# -- add subject
nwb2.subject = pynwb.file.Subject(
    subject_id='animal_01',
    sex='F')

In [ ]:
# -- create NWB 
nwb3 = NWBFile(identifier='nwb_03',
              session_description='',
              session_start_time=datetime.strptime('2019-10-20', '%Y-%m-%d'),
              file_create_date=datetime.now(tzlocal()),
              experimenter='John Smith')
# -- add subject
nwb3.subject = pynwb.file.Subject(
    subject_id='animal_01',
    sex='F')

In [ ]:
NWB.insert([{'nwb_id': 2, 'nwb': nwb2},
            {'nwb_id': 3, 'nwb': nwb3}])

In [ ]:
NWB()

In [ ]:
fetch_nwb3 = (NWB & 'nwb_id=3').fetch1('nwb')

In [ ]:
fetch_nwb3

### Let's also look at the directory where all the NWB files are generated (configured in the `nwbstore`)

In [ ]:
os.listdir(exported_nwb_dir)